In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup,NavigableString, Tag
import time
import requests
import re
import csv
from selenium.webdriver.common.by import By
import pandas as pd
import os

s = Service(r"/usr/local/bin/chromedriver") #使用chromedriver、selenium
driver = webdriver.Chrome(service=s)
    
driver.get("https://icard.ai/") #自動打開網頁（不要關）
    
time.sleep(10) #暫停
    
soup = BeautifulSoup(driver.page_source, 'html.parser') #把script變成美麗湯可讀

results = soup.find_all('div', class_='sc-fkouio-0 hniKmL') 
for result in results:
    
    ranking_title = [] #排行名
    card_name = [] #卡名
    card_link = [] #卡連結
    apply_link = [] #申請連結
    card_img = [] #卡圖片
    card_feature = [] #卡特色 
    
    title = result.find('h3').text
    if title == '2023最新熱門必辦信貸' or  title == '2023信用卡首刷禮推薦' or title == '2023下半年必辦信用卡':
        continue
        
    print(title)
    ranking_title.append(title) #排行名
    cards = result.find_all('div', class_='sc-fkouio-0 bkXcuC')#卡名
    for card in cards:
        name = card.find('div', class_='sc-fkouio-0 flkoQp')
        print(name.text)
        card_name.append(name.text)
        
        link = card.find('div', class_='sc-fkouio-0 eWyXAj')#卡連結
        if link != None:
            link = link.find('a').get('href')
            print('https://icard.ai'+link)
            card_link.append('https://icard.ai'+link)
        else:
            print('無')
            card_link.append('無')
            
        apply = card.find('div', class_='sc-fkouio-0 hTPJxD').find('a').get('href') #申請連結
        print('https://icard.ai'+apply)
        apply_link.append('https://icard.ai'+apply)
        
        c = card.find('div', class_='sc-fkouio-0 sc-17j05y-1 llqmnx kYmowd')#卡圖片
        if c != None:
            img = c.find('img',sizes='100vw').get('src')
            print('https://icard.ai'+img)
            card_img.append('https://icard.ai'+img)
        else:
            print('無')
            card_img.append('無')
            
        result1 =card.find('ul', class_='sc-fkouio-0 sc-1h42shb-2 cjrPow bBDGZz') #卡特色
        result2 = result1.find_all('span')
        text = []
        for re in result2:
            print(re.text)
            text.append(re.text)
        t = '\n'.join(text)
        card_feature.append(t) 
        print('\n')
    print('\n')
    
    cards_dict = {
        '卡名': card_name,
        '卡連結': card_link,
        '卡圖片': card_img,
        '申請連結': apply_link,
        '卡片特色': card_feature    
    }
    
    df = pd.DataFrame(cards_dict)
    
    
    #創資料夾
    
    folder_name = 'icardai'+ranking_title[0]
    path = "/Users/takoyaki/Downloads/專題爬蟲/icardaicsv"  

    folder_path = os.path.join(path, folder_name)
    if not os.path.isdir(folder_path):
        os.makedirs(folder_path, mode=0o777)
    path0 = str(path+'/'+folder_name+"/")
    print(path0)
    df.to_csv(path0+('icardai'+ranking_title[0]+".csv"), index = False)
    
    #driver.close()
    
    #單卡頁面
    for i in range(len(card_name)):
        
        s = Service(r"/usr/local/bin/chromedriver") #使用chromedriver、selenium
        driver = webdriver.Chrome(service=s)
        driver.get(card_link[i]) #自動打開網頁（不要關）
        driver.fullscreen_window()
        time.sleep(3) #暫停
        soup = BeautifulSoup(driver.page_source, 'html.parser') #把script變成美麗湯可讀
        
    #卡名    
    
        result = soup.find('div', class_='sc-fkouio-0 kMjtwV')
        if result ==None:
            continue
        print('★'+result.text+'★')
        name = result.text #卡名
        
    #卡圖片
    
        result = soup.find('div',class_='sc-fkouio-0 sc-17j05y-1 llqmnx jHJsrR')
        if result !=None:
            result = result.find('img')
            img = 'https://icard.ai/'+result.get('src') #卡圖片
            print('卡圖片：\n'+img) 
        else:
            continue
    
        
        
        button= driver.find_element(By.CLASS_NAME, "fpOKLg") #年費
        button.click()
        time.sleep(2)
        
        soup = BeautifulSoup(driver.page_source, 'html.parser') #把script變成美麗湯可讀
        
    #年費彈出框
    
        title = [] #中標
        text1 = [] #內文
        card_apply_link =[] #申請連結
    
        results = soup.find_all('li', class_='sc-1ozxck3-1 llcEFQ')
        for result in results:
            title.append(result.find('div').text[:-1])
            text1.append(result.find('div',class_='sc-1ozxck3-2 eIFtwi').text)
        
        for i in range(len(title)):
            print(title[i])
            print(text1[i])
        
        result = soup.find('div', class_='sc-fkouio-0 iDmBLP')
        if result == None:
            card_apply_link.append('停止申辦')
        else:
            a = result.find('a').get('href')
            card_apply_link.append(a[7:])
            print('\n申請連結：\n'+ card_apply_link[0])
        
        button = driver.find_element(By.CLASS_NAME,'bzgOID') #年費彈出框關起來
        button.click()
        
    #優惠
    
        driver.execute_script("window.scrollBy(0,2400)","") 
        time.sleep(3) #暫停
        
        elements = driver.find_elements(By.CLASS_NAME, "clPheL")#展開
        if len(elements) > 0:
            element = elements[0]  # 取得找到的第一個元素
            element.click()  # 點擊元素
        
        driver.execute_script("window.scrollTo(0, 0);")#滑到頁面頂端
        time.sleep(3) #暫停
        driver.execute_script("window.scrollBy(0,800)","") #自動模擬滑鼠滾輪
        time.sleep(3) #暫停
        
        #用driver找到按鈕 並按下
        buttons = driver.find_elements(By.CLASS_NAME, "eOsKUS") #展開
        for button in buttons:
            if button.is_displayed() and button.is_enabled():
                button.click()
                driver.execute_script("window.scrollBy(0,200)","")
            else:
                #driver.execute_script("window.scrollBy(0,10)","")
                driver.execute_script("window.scrollBy(0,10)","")
                if button.is_displayed() and button.is_enabled():
                    button.click()
                else:
                    driver.execute_script("window.scrollBy(0,10)","")
                    if button.is_displayed() and button.is_enabled():
                        button.click()
                    else:
                        driver.execute_script("window.scrollBy(0,-30)","")
                        if button.is_displayed() and button.is_enabled():
                            button.click()
                        else:
                            driver.execute_script("window.scrollBy(0,50)","") 
                            button.click()
                driver.execute_script("window.scrollBy(0,200)","")
            
        time.sleep(3) #暫停    
        soup = BeautifulSoup(driver.page_source, 'html.parser') #把script變成美麗湯可讀
        #print(soup)
    
        space_list = [] #大標
        results = soup.find_all('div',class_='sc-fkouio-0 eynObR')
        for result1 in results:
            space_list.append(result1.text.strip()) #去前後空格
            #print(result1.text.strip())
        keyword='、'.join(space_list) #把大標當關鍵字
    
        title_list = [] #中標
        text_list = [] #內文
        results = soup.find_all('div',class_='sc-fkouio-0 aJYMs')
        for result1 in results:
            result2 = result1.find('div',class_='sc-fkouio-0 lgyuOQ')#中標
            title_list.append(result2.text)
            result3 = result1.find_all('li',class_='sc-1ozxck3-1 llcEFQ') #商家 詳情
            text = []
            for result4 in result3:
                text.append(result4.text)
                
            text='\n'.join(text)
            #print(type(text))
            text_list.append(text)
        
        for i in range(len(title_list)):
            print('☆'+title_list[i])
            print(text_list[i])
            print('\n')
        
        time.sleep(3) #暫停    
        driver.close()
    
    #裝進dict
    
        card_dict = {
            '卡名':name,
            '圖片':img,
            '申請連結':card_apply_link,
        }
        
        dict1=dict(zip(title, text1))
        dict2 = {
            '優惠標題':title_list,
            '優惠詳情':text_list
        }
        dict3 = {
            '關鍵字':keyword
        }
        
        card_dict.update(dict1)
        #card_dict.update(dict2)
        card_dict.update(dict3)
        
        df = pd.DataFrame(card_dict)
        path1 = "/Users/takoyaki/Downloads/專題爬蟲/icardaicsv/icardai"+ranking_title[0]+"/卡csv/"
        if not os.path.isdir(path1):
            os.makedirs(path1, mode=0o777)
        df.to_csv(path1+'icardai'+name+".csv", index = False)
        
        df = pd.DataFrame(dict2)
        path2 = "/Users/takoyaki/Downloads/專題爬蟲/icardaicsv/icardai"+ranking_title[0]+"/優惠csv/"
        if not os.path.isdir(path2):
            os.makedirs(path2, mode=0o777)
        df.to_csv(path2+'icardai'+name+"優惠.csv", index = False)
        
#driver.close()

2023高現金回饋信用卡
聯邦JCB吉鶴卡
https://icard.ai/card/detail/62dfb0f5e24507765f232f86
https://icard.ai/go?to=https://card.ubot.com.tw/eCard/dspCardOnLineApply.aspx?GID=59&IDE=A&strID=2008060014&PJN=HLF&UTC=CA001
https://icard.ai/_next/image?url=https%3A%2F%2Fcdn.boostus.cc%2Fassets%2F1658830122339-62dfbd12654d1111aad5eaa7&w=1200&q=75
iCard.AI限定! 7/31前新戶滿額享$700
國內一般消費2%無上限
日幣消費3.5%無上限, 日本AP再加碼最優5%


台新玫瑰Giving卡
https://icard.ai/card/detail/5f83c64fe0655f001b2d8a22
https://icard.ai/go?to=https://www.taishinbank.com.tw/eServiceA/CreditCardAP/apply/index.jsp?pc=07
https://icard.ai/_next/image?url=https%3A%2F%2Fcdn.boostus.cc%2Fassets%2F1688136898866-649eac6df4070fdce927c627&w=1200&q=75
透過iCard.AI新戶申辦滿額
享好禮超商$600商品卡+玫瑰粉行李箱
國內假日消費(含保費)、海外消費3%回饋


富邦J卡
https://icard.ai/card/detail/5e15aeca5c386d001d389941
https://icard.ai/go?to=https://ebank.taipeifubon.com.tw/EXT/common/CAC/CACLineIndex.faces?fromSC_code=MA88
https://icard.ai/_next/image?url=https%3A%2F%2Fcdn.boostus.cc%2Fassets%2F1625052459648-60dc55

★台新銀行玫瑰Giving卡★
卡圖片：
https://icard.ai//_next/image?url=https%3A%2F%2Fcdn.boostus.cc%2Fc8d3756e45b644ce8b817869739736d39c31e30e%2F5f83c64fe0655f001b2d8a22.png&w=1200&q=75
申請條件
正卡申請人須年滿20歲，附卡申請人須年滿15歲（未滿20歲者，須由法定代理人「即父母或監護人」為正卡持有人）。
所需文件
1. 正卡及附卡申請人之身分證正反面影本。
2. 上班族：請提供薪資扣繳憑單或近三個月薪資轉帳存摺影本或近三個月鉛印薪資單等。
3. 公司負責人或股東：請提供最近三期營業稅單、個人名下經常往來存摺等影本。
4. 家管及退休人士：請提供財力證明，如定存單、活儲存摺、房屋稅單、地價稅單等。
5. 外籍人士：請另備居留證正反面影本，並徵提臺籍保證人乙名。
首年年費
4500元(有條件免年費), 1. 使用台新電子帳單、行動帳單，即享免年費優惠(若同時使用實體帳單者恕無法參加免年費活動)
2. 如取消行電子帳單、行動帳單或額外申請補印實體帳單，則最晚將於次2月信用卡帳單揭露應繳納之年費。
次年年費
4500元(有條件免年費), 1. 使用台新電子帳單、行動帳單，即享免年費優惠(若同時使用實體帳單者恕無法參加免年費活動)
2. 如取消行電子帳單、行動帳單或額外申請補印實體帳單，則最晚將於次2月信用卡帳單揭露應繳納之年費。
循環利息
6.75%~15%，依電腦評等而定，循環利率之基準日為104年09月01日。
預借現金手續費
1. 每筆預借現金金額之3%，惟不足NT$100者，以NT$100計。另，持卡人辦理電話語音預借現金或網路預借現金，並要求本行將該筆預借現金金額匯入本行以外之金融機構者，需加計收取匯款作業手續費，每筆計收NT$100。
2. 單筆預借現金，日幣提領每月NT$20,000內，非日幣提領每月NT$5,000內，免收預借現金手續費。

申請連結：
https://www.taishinbank.com.tw/eServiceA/CreditCardAP/apply/index.jsp?pc=07
☆刷卡金回饋最高3%，每月上限5,000元(2023/9/30止、限國定例假日)
商家：國內一般消費
詳

★台北富邦J POINTS卡★
卡圖片：
https://icard.ai//_next/image?url=https%3A%2F%2Fcdn.boostus.cc%2Fc8d3756e45b644ce8b817869739736d39c31e30e%2F5e15aeca5c386d001d389941.png&w=1200&q=75
申請條件
正卡：年滿20歲，有正當職業且非在學學生者，年收入達25萬元以上，歡迎檢附財力證明文件申請。A
附卡：年滿15歲，且限正卡申請人之父母、配偶、子女、兄弟姊妹、配偶父母
所需文件
正卡及附卡申請人身分證正反面影本（外籍人士請檢附護照及有效居留證影本）
收入證明文件（擇一提供）：
．薪資單 最近三個月影本
．薪資轉帳/活期存款存摺 存摺影本(有帳號戶名之頁面及近三個月內頁明細)
．股票/基金現值 近一個月資料
．定期存款證明 如：有效存單影本
．不動產謄本
．人壽保單可借貸餘額 近一個月資料且申請人需為要保人
．所得扣繳憑單/所得清單 近一年影本
．建物所有權狀/建物謄本 影本 ．房屋稅單 近一期影本
首年年費
首年免年費
次年年費
1800元(有條件免年費), 次年消費正附卡合併達6萬或12次，享次年免年費優惠。109/12/31前除前述達檻免年費條件外，客戶符合下列任一條件，同享免年費優惠：1. 僅申請電子帳單，2.綁定銀行LINE官方帳號好友，3.本行存戶自扣
循環利息
本行指數型房貸基準利率(I)+加碼利率(該加碼利率區間為0.62%~14.00%，基準日為108.1.1)；年利率上限為15%
預借現金手續費
預借金額×2.5%。其他費用請至http：//www.fubon.com查詢。

申請連結：
https://ebank.taipeifubon.com.tw/EXT/common/CAC/CACLineIndex.faces?fromSC_code=MA88
☆ LINE POINTS 回饋最高1.7%，無上限(2023/12/31止)
商家：國內一般消費
詳情：國內最高1.7%，回饋無上限：國內一般消費1%+綁定富邦數位帳戶自扣本行信用卡費加碼0.7%。


☆ LINE POINTS 回饋最高1%，無上限(2023/12/31止)
商家：國內一般消費
詳情：國內一般消費1%。


☆ LINE POINTS

★中國信託LINE Pay信用卡★
卡圖片：
https://icard.ai//_next/image?url=https%3A%2F%2Fcdn.boostus.cc%2F3685e330ec5277a9dd5661c61f2bc55811f5a628%2F1656486838189-811708-5adb-484f-eb5-7cfea6b32c57&w=1200&q=75
申請條件
1.凡申請正卡者，須年滿20歲，固定年收入：
2.申請託LINE Pay御璽卡/晶緻卡正卡者，須年滿20歲，固定年收入20萬元以上。
3.年滿15歲者可申請附卡。附卡申請人如未滿20歲，須法定代理人(即父母/監護人)共同簽名同意。且須將申請書正本連同身分證正反影本寄回，請勿使用傳真。
4.申請御璽卡/晶緻卡附卡者，正卡人需持有中國信託LINE Pay御璽卡/晶緻卡(可不同版面)。
5.同一卡等(VISA御璽卡、JCB晶緻卡)之申請以1卡為限
所需文件
1. 正卡及附卡申請人之身分證正反面影本。
2. 薪資扣繳憑單或其他財力證明，如近三個月之薪資3. 證明/撥薪存摺、近二個月活儲存摺等影本。
4. 外籍人士請另備居留證資料查詢及護照影本。
5. 大專院校學生除需附上學生證及身分證正反面影本外，另需檢附薪資扣繳憑單、撥薪存摺等財力證明文件。(學生年收入須10萬元以上)。 附註：其他不足部份悉依本行「信用卡個人戶授信準則」之規定辦理。
首年年費
1800元(有條件免年費), 成功綁定LINE Pay或申請本行電子帳單期間，享終身免年費禮遇
次年年費
1800元(有條件免年費), 成功綁定LINE Pay或申請本行電子帳單期間，享終身免年費禮遇
循環利息
本行ARMs指數＋加碼利率(5.97%~13.47%)；上限為15%
預借現金手續費
每筆預借金額X3.5%+150元

申請連結：
https://mkt.ctbcbank.com/recent/202309/NB2023062998/index.html
☆ LINE POINTS 回饋最高1%，無上限(2023/12/31止)
商家：國內一般消費
詳情：一般消費1% LINE POINTS回饋。


☆ LINE POINTS 回饋最高2.2%，每月上限 LINE POINTS 回饋450點(2023/9/30止) +  LINE 

★台北富邦momo卡★
卡圖片：
https://icard.ai//_next/image?url=https%3A%2F%2Fcdn.boostus.cc%2Fc8d3756e45b644ce8b817869739736d39c31e30e%2F5e15aed95c386d001d389b45.png&w=1200&q=75
申請條件
正卡：年滿20歲，有正當職業且非在學學生者。
附卡：年滿15歲，且限正卡申請人之父母、配偶、子女、兄弟姊妹、配偶父母
御璽/鈦金/晶緻卡：年收入達20萬元以上，歡迎檢附財力證明文件申請。
所需文件
財力證明文件
首年年費
1800元(有條件免年費), 符合下列任一條件，即享免年費優惠
•申請電子帳單免年費
•momo通路不限金額任刷一筆
•成功申請電子帳單且不寄送實體帳單
•綁定台北富邦銀行LINE官方帳號好友
•使用台北富邦銀行帳號自動扣繳富邦信用卡帳單
次年年費
1800元(有條件免年費), 次年消費正附卡合併達6萬或12次
循環利息
本行指數型房貸基準利率(I)+加碼利率(該加碼利率區間為0.62%~14.00%，基準日為108.1.1
預借現金手續費
預借金額×2.5%

申請連結：
https://ebank.taipeifubon.com.tw/EXT/common/CAC/PLCCIndex.faces?fromSC_code=MA88
☆momo幣回饋最高1%，無上限(2023/12/31止)
商家：國內一般消費、海外一般消費
詳情：一般消費享1%回饋無上限。以上回饋為momo幣。
*momo通路含購物網、電視購物、型錄，全站商品皆適用，特殊商品除外，請依商品頁標示為主。
*momo旅遊商品(富昇旅行社)、momo摩天商城為一般消費。
*使用第三方支付平台(如LINE Pay、街口支付)之交易(含LINE Pay綁定momo卡於momo購物網消費)皆視為店外一般消費回饋1%。


☆momo幣回饋最高1%，無上限(2023/12/31止)
商家：國內一般消費、海外一般消費
詳情：一般消費享1%回饋無上限。以上回饋為momo幣。
*momo通路含購物網、電視購物、型錄，全站商品皆適用，特殊商品除外，請依商品頁標示為主。
*momo旅遊商品(富昇旅行社)、momo摩天商城為一般消費。
*使用第三方支付平台(如LINE 

...

★台新銀行@GoGo悠遊御璽卡★
卡圖片：
https://icard.ai//_next/image?url=https%3A%2F%2Fcdn.boostus.cc%2F3685e330ec5277a9dd5661c61f2bc55811f5a628%2F1676952071670-fc101-88a0-880-413f-8cdd3d20d78&w=1200&q=75
申請條件
正卡申請人須年滿20歲；附卡申請人須年滿15歲（未滿20歲者，須由法定代理人「即父母或監護人」為正卡持有人）。
所需文件
1. 正卡及附卡申請人之身分證正反面影本。
2. 上班族：請提供薪資扣繳憑單或近三個月薪資轉帳存摺影本或近三個月鉛印薪資單等。
3. 公司負責人或股東：請提供最近三期營業稅單、個人名下經常往來存摺等影本。
4. 家管及退休人士：請提供財力證明，如定存單、活儲存摺、房屋稅單、地價稅單等。
5. 外籍人士：請另備居留證正反面影本，並徵提臺籍保證人乙名。
首年年費
3000元(有條件免年費), 核卡後申辦以下指定項目其中任二項且持卡期間未終止該服務，即享「活動期間免年費」優惠(活動期間：2021/1/1~2021/12/31)，指定項目如下(四選二)：
1. 下載信用卡Richart Life APP，且完成登入。
2. 申請信用卡行動/電子帳單。
3. 使用台新信用卡辦理公用事業費用自動轉帳代扣繳(註：公用事業費用包含代繳水費、電費、瓦斯費、電信費用、健保費用)。
4. 使用台新銀行帳戶自動扣繳信用卡款。
次年年費
3000元(有條件免年費), 年度累積消費滿NT$6萬或年度消費次數達6次，可折抵正卡次年年費及正卡名下之所有附卡年費，其餘免年費優惠活動依照官網公告為主。
循環利息
6.75%~15%，依電腦評等而定，循環利率之基準日為104年09月01日。
預借現金手續費
1. 每筆預借現金金額之3%，惟不足NT$100者，以NT$100計。另，持卡人辦理電話語音預借現金或網路預借現金，並要求本行將該筆預借現金金額匯入本行以外之金融機構者，需加計收取匯款作業手續費，每筆計收NT$100。
2. 單筆預借現金，日幣提領每月NT$20,000內，非日幣提領每月NT$5,000內，免收預借現金手續費。

申請連結：
https://www.taishinbank.com.tw/

2023加油信用卡
中信中油聯名卡
https://icard.ai/go?to=https://aurora.icard.ai/card/detail/5e15aebf5c386d001d38980a
https://icard.ai/go?to=https://mkt.ctbcbank.com/recent/202311/NB2023062172-15_1/index.html
https://icard.ai/_next/image?url=https%3A%2F%2Fcdn.boostus.cc%2Fassets%2F1688374320892-64a28b21ec213f54dd0f4cd8&w=1200&q=75
中油加油最優6.8%回饋
加油消費同時累積中油會員點數及中信紅利點數


富邦OPEN Possible卡
https://icard.ai/cards/bank/channel_concept%2F5dccdfad82d7a71059d9065b
https://icard.ai/go?to=https://apply.taipeifubon.com.tw/PLCC/TWM/Index?fromSC_code=MA88
https://icard.ai/_next/image?url=https%3A%2F%2Fcdn.boostus.cc%2Fassets%2F1688629383800-611247d972abdd1efc9ea46e&w=1200&q=75
電信費/加油5.5%回饋、回饋上限最高$600/月
9/30前申辦新戶最優$1,000
myfone門市/購物最高10%回饋


台新FlyGo卡
https://icard.ai/card/detail/5e15aebc5c386d001d38976f?rid=channel_concept%2F5d6ce3288a0112506b5ed6a9
https://icard.ai/go?to=https://www.taishinbank.com.tw/eServiceA/CreditCardAP/apply/index.jsp?pc=07
https://icard.ai/_next/image?url=https%3A%2F%2Fcdn.boostus.cc%2Fassets%2

★台北富邦J POINTS卡★
★台新銀行FlyGo卡★
卡圖片：
https://icard.ai//_next/image?url=https%3A%2F%2Fcdn.boostus.cc%2F3685e330ec5277a9dd5661c61f2bc55811f5a628%2F1669787388564-14305d-2c5-67ab-eedf-08d0ebef703&w=1200&q=75
申請條件
正卡申請人須年滿20歲；附卡申請人須年滿15歲（未滿20歲者，須由法定代理人「即父母或監護人」為正卡持有人）。
所需文件
1. 正卡及附卡申請人之身分證正反面影本。
2. 上班族：請提供薪資扣繳憑單或近三個月薪資轉帳存摺影本或近三個月鉛印薪資單等。
3. 公司負責人或股東：請提供最近三期營業稅單、個人名下經常往來存摺等影本。
4. 家管及退休人士：請提供財力證明，如定存單、活儲存摺、房屋稅單、地價稅單等。
5. 外籍人士：請另備居留證正反面影本，並徵提臺籍保證人乙名。

申請人同意於申辦任何等級之信用卡別時，台新銀行得重新評估信用狀況，調整申請人之信用卡額度。
首年年費
4500元(有條件免年費), 使用行動帳單/電子帳單，即享免年費優惠(同時使用實體帳單，恕無法參加免年費活動)
次年年費
4500元(有條件免年費), 使用台新行動帳單/電子帳單，即享次年免年費優惠(同時使用實體帳單，恕無法參加免年費活動)
循環利息
6.75%~15%，依電腦評等而定，循環利率之基準日為104年09月01日。
預借現金手續費
1. 每筆預借現金金額之3%，惟不足NT$100者，以NT$100計。另，持卡人辦理電話語音預借現金或網路預借現金，並要求本行將該筆預借現金金額匯入本行以外之金融機構者，需加計收取匯款作業手續費，每筆計收NT$100。
2. 單筆預借現金，日幣提領每月NT$20,000內，非日幣提領每月NT$5,000內，免收預借現金手續費。

申請連結：
https://www.taishinbank.com.tw/eServiceA/CreditCardAP/apply/index.jsp?pc=07
☆現金回饋最高0.5%，無上限(2024/3/31止)
商家：國內一般消費
詳情：2023/4/1~2024/3/31結帳帳單
一般消費 0.5%無上限。


☆現金回

★玉山銀行商務御璽卡★
卡圖片：
https://icard.ai//_next/image?url=https%3A%2F%2Fcdn.boostus.cc%2Fc8d3756e45b644ce8b817869739736d39c31e30e%2F5e15aec35c386d001d389839.png&w=1200&q=75
申請條件
1. 正卡：年滿18歲皆可申請，並提供相關財力證明。
2. 附卡：須年滿15歲且為正卡之配偶、父母、子女、兄弟姊妹及配偶之父母等親屬關係。
所需文件
1. 身分證正、反面影本。
2. 正卡申請人財力證明文件(請提供下列任一項)：
(1) 薪資單-最近三個月影本。
(2) 薪資轉帳/活期存款存摺-存摺影本(有帳號戶名之頁面及近三個月內頁明細)。
(3) 股票/基金現值-近一個月資料。
(4) 定期存款證明。
(5) 不動產謄本。
(6) 人壽保單可借貸餘額-近一個月資料且申請人需為要保人。
(7) 建物所有權狀/建物謄本-影本。
(8) 房屋稅單-近一期影本。
首年年費
首年免年費。
次年年費
3000元(有條件免年費), 1. 使用帳單e化及玉山帳戶自動扣繳信用卡款期間享免年費優惠。
2. 正附卡合併年度消費滿15萬元(含)以上。
循環利息
5.88%~15%。
預借現金手續費
預借現金金額x3.5% + (新臺幣100元/3.5美元/350日圓/2.5歐元)。

申請連結：
https://esun.co/RA5fk
☆新戶享刷卡金300元(2023/12/31止) + 新卡友享刷卡金200元(2023/9/30止)
詳情：送刷卡金。【1】新戶首次申辦玉山商務御璽卡正卡且核卡45天內新增一般消費累積達5,000元，贈【刷卡金300元】。
【2】不限新舊戶，首次申辦玉山商務御璽卡正卡，且完成以上首刷禮條件，加碼贈「刷卡金200元」，正附卡合併計算。
條件：核卡45天內於指定通路有消費滿5000元


☆新卡友享刷卡金200元(2023/9/30止)
詳情：送刷卡金。不限新舊戶首次申辦玉山商務御璽卡正卡，且核卡45天內新增一般消費累積達5,000元即加碼贈【刷卡金200元】，正附卡合併計算。
條件：核卡45天內於指定通路有消費滿5000元


☆新會員享 OPENPOINTS 50元(2023/9/30止)
詳情：送 OPENPOI

WebDriverException: Message: disconnected: not connected to DevTools
  (failed to check if window was closed: disconnected: not connected to DevTools)
  (Session info: chrome=114.0.5735.198)
Stacktrace:
0   chromedriver                        0x0000000104ced598 chromedriver + 4973976
1   chromedriver                        0x0000000104ce4913 chromedriver + 4938003
2   chromedriver                        0x00000001048a1cd6 chromedriver + 470230
3   chromedriver                        0x0000000104888a61 chromedriver + 367201
4   chromedriver                        0x00000001048887c3 chromedriver + 366531
5   chromedriver                        0x00000001048a3862 chromedriver + 477282
6   chromedriver                        0x0000000104923f4e chromedriver + 1003342
7   chromedriver                        0x0000000104909793 chromedriver + 894867
8   chromedriver                        0x00000001048d721f chromedriver + 688671
9   chromedriver                        0x00000001048d856e chrome